In [89]:
import sys
sys.path.insert(0, r'C:\Users\HARSH\Documents\chatweb3-backend')
from utils import *
set_api_key()

In [90]:
import re
import requests

from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.prompts.base import BaseOutputParser


TEMPLATE = \
'''
Query: {query}
Note: The query is related to the blockchain ecosystem.
Generate 10 related searches most asked on the web. 
1.'''

class ChatOutputParser(BaseOutputParser):

    @property
    def _type(self) -> str:
        """Return the type key."""
        return "chat_output_parser"

    def parse(self, text: str) -> str:
        """Parse the output of an LLM call."""
        text = text.strip()
        text = re.sub(r"\d+. ", '', text)
        text = text.split('\n')
        return text

In [91]:

class SearchURLs():
    def __init__(self,) -> None:
        super().__init__()
        llm = OpenAI(
            temperature=1.0, max_tokens=-1,
        )
        output_parser = ChatOutputParser()
        relSearch_prompt = PromptTemplate(
            input_variables=["query"],
            template=TEMPLATE,
            output_parser=output_parser,
        )
        self.chain = LLMChain(llm=llm, prompt=relSearch_prompt, verbose=False)

        subscription_key="df01e65a4da8434b841e8a81b34c9d25"
        self.search_url = "https://api.bing.microsoft.com/v7.0/search"
        self.headers = {"Ocp-Apim-Subscription-Key" : subscription_key}

    def via_llm(self, query): 
        relSearches = self.chain.apply_and_parse([{"query": query}])[0]
        return relSearches

    def via_bing(self, query):
        query = query+" Note: The query is related to the blockchain ecosystem."
        params  = {"q": query, "textDecorations": True, "textFormat": "HTML"}
        response = requests.get(self.search_url, headers=self.headers, params=params)
        response.raise_for_status()
        response = response.json()
        urls = [x['url'] for x in response['webPages']['value']]
        try: relSearches = [r['text'] for r in response['relatedSearches']['value']]
        except: relSearches = []
        return urls, relSearches

In [92]:
with open("basic_questions.txt") as file:
    queries = [line.rstrip() for line in file]

In [93]:
def find_urls(query, searcher, depth):
    urls, relS_bing = searcher.via_bing(query)
    relS_llm = searcher.via_llm(query)
    print({query: urls})
    if depth==0: return {query: urls}
    relS = relS_bing + relS_llm
    q_urls = []
    for q in relS:
        x = find_urls(q, searcher, depth=depth-1)
        q_urls.append(x)
    return q_urls

In [94]:
searcher = SearchURLs()
for q in queries:
    urls = find_urls(q, searcher, depth=1)
    break

{'What is web3?': ['https://hbr.org/2022/05/what-is-web3', 'https://ethereum.org/en/web3/', 'https://www.gemini.com/cryptopedia/what-is-web3-explained-blockchain-and-web3-crypto-projects', 'https://www.nytimes.com/interactive/2022/03/18/technology/web3-definition-internet.html', 'https://chain.link/education/web3', 'https://www.forbes.com/sites/bernardmarr/2022/01/24/what-is-web3-all-about-an-easy-explanation-with-examples/', 'https://byjus.com/current-affairs/web-3-0/', 'https://consensys.net/blog/blockchain-explained/what-is-web3-here-are-some-ways-to-explain-it-to-a-friend/', 'https://www.geeksforgeeks.org/what-is-blockchain-ecosystem/', 'https://crypto-current.co/what-are-web3-ecosystems/']}
{'What is the purpose of web3?': ['https://hbr.org/2022/05/what-is-web3', 'https://ethereum.org/en/web3/', 'https://en.wikipedia.org/wiki/Web3', 'https://www.nytimes.com/interactive/2022/03/18/technology/web3-definition-internet.html', 'https://www.pcmag.com/how-to/what-is-web3-and-how-will-it-

Retrying langchain.llms.openai.BaseOpenAI.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised RateLimitError: Rate limit reached for default-text-davinci-003 in organization org-mrK2kSCEDDLFwshwzA1wjEBX on tokens per min. Limit: 250000 / min. Current: 324680 / min. Contact support@openai.com if you continue to have issues..


{'What is the role of web3 in blockchain?': ['https://101blockchains.com/blockchain-in-web-3-0/', 'https://hbr.org/2022/05/what-is-web3', 'https://blog.gaurav-dhiman.com/what-is-web3-and-role-of-blockchain-and-crypto-in-it', 'https://www.blockchain-council.org/blockchain/blockchain-technology-in-web-3-0/', 'https://www.gemini.com/cryptopedia/what-is-web3-explained-blockchain-and-web3-crypto-projects', 'https://academy.binance.com/en/articles/what-s-the-relationship-between-blockchain-and-web3', 'https://blockchain-comparison.com/blockchain-knowledge-base/web-3/', 'https://www.cloudflare.com/learning/insights-how-blockchain-web3/', 'https://101blockchains.com/flux-web3/', 'https://www.geeksforgeeks.org/what-is-blockchain-ecosystem/']}
{'What is the web3 JavaScript API?': ['https://web3js.org/', 'https://moralis.io/web3-and-javascript-what-is-javascript-and-web3-js/', 'https://web3.hashnode.com/what-is-web3js-an-introduction-into-the-web3js-libraries', 'https://stackoverflow.com/question

In [95]:
urls

[{'What is the purpose of web3?': ['https://hbr.org/2022/05/what-is-web3',
   'https://ethereum.org/en/web3/',
   'https://en.wikipedia.org/wiki/Web3',
   'https://www.nytimes.com/interactive/2022/03/18/technology/web3-definition-internet.html',
   'https://www.pcmag.com/how-to/what-is-web3-and-how-will-it-work',
   'https://www.thedrum.com/opinion/2022/02/09/what-web3-and-why-does-it-matter',
   'https://www.gemini.com/cryptopedia/what-is-web3-explained-blockchain-and-web3-crypto-projects',
   'https://www.drishtiias.com/daily-updates/daily-news-analysis/web-3-0',
   'https://byjus.com/current-affairs/web-3-0/',
   'https://consensys.net/blog/blockchain-explained/what-is-web3-here-are-some-ways-to-explain-it-to-a-friend/']},
 {'What is the role of web3 in blockchain?': ['https://101blockchains.com/blockchain-in-web-3-0/',
   'https://hbr.org/2022/05/what-is-web3',
   'https://blog.gaurav-dhiman.com/what-is-web3-and-role-of-blockchain-and-crypto-in-it',
   'https://www.blockchain-counc